In [1]:
#!/usr/bin/env python3

import numpy as np
import os
from scipy import interpolate, signal
import filecmp
from shutil import copyfile
import sys
from PyLTSpice.LTSpice_RawRead import RawRead

def apply_ltspice_neuron(simname,sig_in_x,sig_in_y,**kwargs):
  
    verbose = kwargs.get("verbose",False)
    interpol = kwargs.get("interpolate",True)
  
    default_ltspice_command = "C:\Program Files\LTC\LTspiceXVII\XVIIx64.exe -Run -b " 
    if sys.platform == "linux":
        default_ltspice_command = 'wine C:\\\\Program\\ Files\\\\LTC\\\\LTspiceXVII\\\\XVIIx64.exe -Run -b '
  
    ltspice_command = kwargs.get("ltspice_command",default_ltspice_command)
  
  

    params  = kwargs.get("params",{})

    simname = simname.replace(".asc","")

    with open("sig_in.csv_","w") as f:
        for i in range(0,len(sig_in_y)):
          f.write("{:E}\t{:E}\n".format(sig_in_x[i],sig_in_y[i]))
        f.close()
    
    with open("trancmd.txt_","w") as f:
        f.write(".param transtop {:E}\n".format(sig_in_x[-1]-sig_in_x[0]))
        f.write(".param transtart {:E}\n".format(sig_in_x[0]))
        f.write(".param timestep {:E}\n".format(sig_in_x[1]-sig_in_x[0]))
        f.close()

    with open("param.txt_","w") as f:
        for key in params:
            f.write(".param {:s} {:E}\n".format( key,params[key]  ))
        f.write("\n")
        f.close()
    

    sth_changed = False

  # check if we ran the simulation before with exact same input, can save time
    if os.path.isfile('sig_in.csv') and filecmp.cmp('sig_in.csv_', 'sig_in.csv') :
        pass
#         print("sig_in.csv has not changed")
    else:
        sth_changed = True
        copyfile('sig_in.csv_', 'sig_in.csv')
    
    if os.path.isfile('trancmd.txt') and filecmp.cmp('trancmd.txt_', 'trancmd.txt'): 
        pass
#         print("trancmd.txt has not changed")
    else:
        sth_changed = True
        copyfile('trancmd.txt_', 'trancmd.txt')
    
    if os.path.isfile('param.txt') and filecmp.cmp('param.txt_','param.txt') : 
        pass
#         print("param.txt has not changed")
    else:
        sth_changed = True
        copyfile('param.txt_','param.txt')
    
    
    if os.path.isfile("{:s}.raw".format(simname)): ## raw file already exists
    # get rawfile modification date
        rawmdate = os.path.getmtime("{:s}.raw".format(simname))
    # get ascfile modification date
        ascmdate = os.path.getmtime("{:s}.asc".format(simname))
        if ascmdate > rawmdate: # asc file has been modified in the meantime
   #   print("{:s}.asc is newer than {:s}.raw".format(simname,simname))
              sth_changed = True
        else:
             pass
#                 print("{:s}.asc is older than {:s}.raw".format(simname,simname))
    else:
        sth_changed = True

  # do not execute ltspice if nothing has changed
    if sth_changed:
    #print("executing ./wine_ltspice.sh, saving STDOUT to wine_ltspice.log")
    #os.system("{:s} {:s}.asc > wine_ltspice.log 2>&1".format(simname))
        if sys.platform == "linux":
            os.system(ltspice_command+" {:s}.asc".format(simname))
        else:
            import subprocess
            subprocess.run([*ltspice_command.split(), "{:s}.asc".format(simname)])
    
    else:
        pass
#         print("input data did not change, reading existing .raw file")
    
    ltr = RawRead("{:s}.raw".format(simname))
  
  
    if verbose:
        for name in ltr.get_trace_names():
            for step in ltr.get_steps():
                tr = ltr.get_trace(name)
                print(name)
                print('step {:d} {}'.format(step, tr.get_wave(step)))
  
  #os.system("./clean_up.sh")
    os.remove("param.txt_")
    os.remove("trancmd.txt_")
    os.remove("sig_in.csv_")
  
    IR1 = ltr.get_trace("V(vout)")
    x = ltr.get_trace("time") 
  
  #  #### the abs() is a quick and dirty fix for some strange sign decoding errors
    vout_x = abs(x.get_wave(0))
    vout_y = IR1.get_wave(0)
 
  #  interpolate ltspice output, so you have the same x value spacing as in the input voltage vector
    if interpol:
        f = interpolate.interp1d(vout_x,vout_y)
        vout_x = sig_in_x
        vout_y = f(sig_in_x)
  
    return (vout_x,vout_y)

def apply_ltspice_synapse(simname,sig_in_x,sig_in_y1,sig_in_y2,**kwargs):
  
    verbose = kwargs.get("verbose",False)
    interpol = kwargs.get("interpolate",True)
  
    default_ltspice_command = "C:\Program Files\LTC\LTspiceXVII\XVIIx64.exe -Run -b " 
    if sys.platform == "linux":
        default_ltspice_command = 'wine C:\\\\Program\\ Files\\\\LTC\\\\LTspiceXVII\\\\XVIIx64.exe -Run -b '
  
    ltspice_command = kwargs.get("ltspice_command",default_ltspice_command)
  
  

    params  = kwargs.get("params",{})

    simname = simname.replace(".asc","")

    with open("sig1_in.csv_","w") as f:
        for i in range(0,len(sig_in_y1)):
            f.write("{:E}\t{:E}\n".format(sig_in_x[i],sig_in_y1[i]))
        f.close()
    
    with open("sig2_in.csv_","w") as f:
        for i in range(0,len(sig_in_y2)):
            f.write("{:E}\t{:E}\n".format(sig_in_x[i],sig_in_y2[i]))
        f.close()
    
    with open("trancmd.txt_","w") as f:
        f.write(".param transtop {:E}\n".format(sig_in_x[-1]-sig_in_x[0]))
        f.write(".param transtart {:E}\n".format(sig_in_x[0]))
        f.write(".param timestep {:E}\n".format(sig_in_x[1]-sig_in_x[0]))
        f.close()

    with open("param.txt_","w") as f:
        for key in params:
            f.write(".param {:s} {:E}\n".format( key,params[key]  ))
        f.write("\n")
        f.close()
    

    sth_changed = False

  # check if we ran the simulation before with exact same input, can save time
    if os.path.isfile('sig1_in.csv') and filecmp.cmp('sig1_in.csv_', 'sig1_in.csv') :
        pass
        #         print("sig1_in.csv has not changed")
    else:
        sth_changed = True
        copyfile('sig1_in.csv_', 'sig1_in.csv')
    
    if os.path.isfile('sig2_in.csv') and filecmp.cmp('sig2_in.csv_', 'sig2_in.csv') :
        pass
#         print("sig2_in.csv has not changed")
    else:
        sth_changed = True
        copyfile('sig2_in.csv_', 'sig2_in.csv')    
    
    if os.path.isfile('trancmd.txt') and filecmp.cmp('trancmd.txt_', 'trancmd.txt'): 
        pass
#         print("trancmd.txt has not changed")
    else:
        sth_changed = True
        copyfile('trancmd.txt_', 'trancmd.txt')
    
    if os.path.isfile('param.txt') and filecmp.cmp('param.txt_','param.txt') : 
        pass
#         print("param.txt has not changed")
    else:
        sth_changed = True
        copyfile('param.txt_','param.txt')
    
    
    if os.path.isfile("{:s}.raw".format(simname)): ## raw file already exists
    # get rawfile modification date
        rawmdate = os.path.getmtime("{:s}.raw".format(simname))
    # get ascfile modification date
        ascmdate = os.path.getmtime("{:s}.asc".format(simname))
        if ascmdate > rawmdate: # asc file has been modified in the meantime
            pass
#             print("{:s}.asc is newer than {:s}.raw".format(simname,simname))
            sth_changed = True
        else:
            pass
#             print("{:s}.asc is older than {:s}.raw".format(simname,simname))
    else :
        sth_changed = True

  # do not execute ltspice if nothing has changed
    if sth_changed:
    #print("executing ./wine_ltspice.sh, saving STDOUT to wine_ltspice.log")
    #os.system("{:s} {:s}.asc > wine_ltspice.log 2>&1".format(simname))
        if sys.platform == "linux":
            os.system(ltspice_command+" {:s}.asc".format(simname))
        else:
            import subprocess
            subprocess.run([*ltspice_command.split(), "{:s}.asc".format(simname)])
    
    else:
        pass
#         print("input data did not change, reading existing .raw file")
    
    ltr = RawRead("{:s}.raw".format(simname))
  
  
    if verbose:
        for name in ltr.get_trace_names():
            for step in ltr.get_steps():
                tr = ltr.get_trace(name)
                print(name)
                print('step {:d} {}'.format(step, tr.get_wave(step)))
  
  #os.system("./clean_up.sh")
    os.remove("param.txt_")
    os.remove("trancmd.txt_")
    os.remove("sig1_in.csv_")
    os.remove("sig2_in.csv_")  
  
    IR1 = ltr.get_trace("V(vout)")
    x = ltr.get_trace("time") 
  
  #  #### the abs() is a quick and dirty fix for some strange sign decoding errors
    vout_x = abs(x.get_wave(0))
    vout_y = IR1.get_wave(0)
 
  #  interpolate ltspice output, so you have the same x value spacing as in the input voltage vector
    if interpol:
        f = interpolate.interp1d(vout_x,vout_y)
        vout_x = sig_in_x
        vout_y = f(sig_in_x)
  
    return (vout_x,vout_y)


Found Numpy. WIll be used for storing data
